# Computer Networks Computer Assignment 2
## Arman Rostami, Parsa Ghorbani

### Initialization

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
SIMULATION_COUNT = 1
TCL_FILENAME = "congestion.tcl"
CWND_FILENAME_1 = "cwnd1.csv"
CWND_FILENAME_2 = "cwnd2.csv"
GOODPUT_FILENAME_1 = "goodput1.csv"
GOODPUT_FILENAME_2 = "goodput2.csv"
RTT_FILENAME_1 = "rtt1.csv"
RTT_FILENAME_1 = "rtt2.csv"

In [3]:
plt.rcParams["figure.figsize"] = (14, 5) 

### Simulator

In [4]:
class Simulator:
    def __init__(self, congestion_algorithm):
        self.congestion_algorithm = congestion_algorithm
        
    def plot_linear(self, x, y):
        plt.plot(x, y)
        plt.xlabel(x.name)
        plt.ylabel(y.name)
        plt.title("%s - %s Relationship" % (x.name, y.name))

In [6]:
def train(self):
    for i in range(0, SIMULATION_COUNT):
        os.system("ns " + TCL_FILENAME + " " + self.congestion_algorithm)
        
Simulator.train = train

### Train Simulators

In [7]:
newRenoSimulator = Simulator("Newreno")
newRenoSimulator.train()

In [8]:
tahoeSimulator = Simulator("Tahoe")
tahoeSimulator.train()

In [9]:
vegasSimulator = Simulator("Vegas")
vegasSimulator.train()